# Content:¶
Goal of this notebook is to prepare the analytical base table (ABT) for all future analysis. Therefore the sollwoing steps are necessary
1. Built a mastertable for IHS and SE based on a unique kex (here the partNumber)
2. Load the attribute Table from HANA
3. Join every cleaned attribute table to the master table (ABT)

The final ABT may look be very wide. However it is expected to have significant overlap between columns and also by throwing away uninformative columns the final number of meaningful columns should be decreased drastically. 

In [1]:
# Import packages (necessary for nbconvert runs)
%run packages.py
if globals()['pd']: 
    print("start script successfully loaded all packages")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


start script successfully loaded all packages


In [2]:
from src.udf import create_att_table
from src.udf import clean_raw_tables
from src.udf import make_read_data_url
from src.udf import create_att_table

In [3]:
import re
import itertools
# import pickle
import multiprocessing as mp
from tqdm import tqdm
import os
import sys
import requests
from itertools import product
from requests.auth import HTTPBasicAuth
import json
from collections import Counter
# Add source script to read data from Hana
sys.path.append('/home/Z000SYPA/hana-r-python-connecticity/python/')

In [4]:
src_path = os.path.join(os.path.dirname(os.getcwd()), "src")
#packages = os.path.join(src_path, "packages.py")
print(src_path)

/home/ubuntu/matspec_ai/src


## Set global variables

In [5]:
# GLOBAL VARIABLES
prod_run = True if ('prod' in os.popen('hostname').read().rstrip()[-4:]) else False # Checks if running on prod instance
hana_sys = '1' # Can either be '0', 'q' or '1'
use_cache = True # If True, then intermediate results are stored/read on/from S3
writeback_to_hana = True

datapath = os.path.join(os.path.dirname(os.getcwd()), "data")
print(datapath)
if not os.path.exists(datapath):
    os.makedirs(datapath)

/home/ubuntu/matspec_ai/data


In [6]:
if (os.path.exists('/S3/FILES_MT_S3')):
    os.system('sudo chown -R ubuntu:iam-synced-users /S3/FILES_MT_S3')
    os.system('sudo chmod -R 777 /S3/FILES_MT_S3')

## Define odata HANA settings

In [7]:
# Get username and password for HANA
HANA_username = ("%s" % os.popen('/opt/parameter-get user').read()).rstrip('\n')
if (hana_sys == '0'):
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh0').read()).rstrip('\n')
else:
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh1').read()).rstrip('\n')

In [8]:
# Read parameters

# Create URL to CV
if (hana_sys == '1'):
    main_url = 'https://hanadatalake.siemens.com/siemens/MATSPEC/AWS/'
else:
    main_url = 'https://hanadatalake-eh' + hana_sys + '.siemens.com/siemens/MATSPEC/AWS/'

xsodata_hana_to_py = 'ODATA_CONNECTIONS.xsodata'
#source_object = '/T_IHSDOC'

## Read data from HANA

In [10]:
data_to_read = ['/V_MASTERDATA_PARTS_ATTRIBUTES', '/V_MASTERDATA_PARTS']

In [11]:
odata_filter = ["&$filter=substring(EXTERNAL_ID,1,1) eq \'" + str(k) +"\'" for k in range(0,10)]
source_filter = ["DATA_SOURCE_ID eq \'SE\'", "DATA_SOURCE_ID eq \'IHS\'"]
null_filter = ["ATTRIBUTE_VALUE ne \'Null\'"]
statements = list(product(odata_filter, source_filter, null_filter))

In [12]:
get_data_dict = dict(zip(data_to_read,[statements,[]]))
#get_data_dict = {'/V_MASTERDATA_PARTS': []}

In [13]:
get_data_dict.keys()

dict_keys(['/V_MASTERDATA_PARTS_ATTRIBUTES', '/V_MASTERDATA_PARTS'])

In [14]:
df_dict = dict()
for dataset, statements in get_data_dict.items(): 
        
    print(dataset)
    if len(statements) > 0:
        df_list = []
       # break
        for statement in tqdm(statements): 
            url = make_read_data_url(main_url, xsodata_hana_to_py,dataset , statement)
            #print(url)
            result = requests.get(url=url, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
            data = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')
            #df_list.append(data.drop(columns = ['GenID','__metadata'], inplace = True))
            df_list.append(data)
        df_dict["df_" + dataset[1:]] = pd.concat(df_list)
    else: 
        url = make_read_data_url(main_url, xsodata_hana_to_py,dataset)
        result = requests.get(url=url, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
        df = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')
        df.drop(columns = ['GenID','__metadata'], inplace = True)
        df_dict["df_" + dataset[1:]] = df

  0%|          | 0/20 [00:00<?, ?it/s]

/V_MASTERDATA_PARTS_ATTRIBUTES


/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  5%|▌         | 1/20 [00:31<10:07, 31.98s/it]/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
 10%|█         | 2/20 [01:56<14:20, 47.80s/it]/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureReq

/V_MASTERDATA_PARTS


/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [15]:
np.round(sys.getsizeof(df_dict["df_V_MASTERDATA_PARTS"])/1e9,2)

0.65

In [16]:
df_dict["df_V_MASTERDATA_PARTS"].loc[df_dict["df_V_MASTERDATA_PARTS"].PART_ID.str.contains("\?") == True]

Empty DataFrame
Columns: [PART_ID, EXTERNAL_ID, PART_DESCRIPTION, CATEGORY_NAME, MANUFACTURER_NAME, DATASHEET_URL, SUPPLIER_DATASHEET_URL, TAXONOMY_PATH, ESTIMATED_END_OF_LIFE, LIFE_CYCLE_STAGE, REACH_DATA_URL, DATA_SOURCE_ID]
Index: []

In [17]:
df_att = df_dict["df_V_MASTERDATA_PARTS_ATTRIBUTES"].drop(columns=['GenID','__metadata', "DATA_SOURCE_ID", "EXTERNAL_ID"])

In [18]:
round(sys.getsizeof(df_att)/1e9,2)

4.57

In [19]:
df_att.head()

PART_ID             ATTRIBUTE_NAME  \
0  00012520                     Acting   
1  00012520                Application   
2  00012520             Current Rating   
3  00012520              Fuse Material   
4  00012520  Maximum AC Voltage Rating   

                                     ATTRIBUTE_VALUE ATTRIBUTE_UOM  
0                                          Slow Blow                
1  Power Supply Adapter|Primary Protection on PCB...                
2                                                2.5             A  
3                                            Ceramic                
4                                                250             V

In [20]:
df_master = df_dict["df_V_MASTERDATA_PARTS"]
df_master.head()

PART_ID  EXTERNAL_ID                                   PART_DESCRIPTION  \
0  00011006   1663636199  Electric Fuse, Very Fast Blow, 1.6A, 250VAC, 1...   
1  00011006     29279700  Fuse Miniature Fast Acting 1.6A 250V Holder Ca...   
2  00011006     29279700  Fuse Miniature Fast Acting 1.6A 250V Holder Ca...   
3  00011006     29279700  Fuse Miniature Fast Acting 1.6A 250V Holder Ca...   
4  00011006     29279700  Fuse Miniature Fast Acting 1.6A 250V Holder Ca...   

  CATEGORY_NAME MANUFACTURER_NAME  \
0         Fuses          Schurter   
1         Fuses          Schurter   
2         Fuses          Schurter   
3         Fuses          Schurter   
4         Fuses          Schurter   

                                       DATASHEET_URL  \
0  https://4donline.ihs.com/images/VipMasterIC/IC...   
1  http://download.siliconexpert.com/pdfs2/2019/2...   
2  http://download.siliconexpert.com/pdfs2/2019/2...   
3  http://download.siliconexpert.com/pdfs2/2019/2...   
4  http://download.siliconexpert.com/pdfs2/2019/2...   

                              SUPPLIER_DATASHEET_URL  \
0                                               None   
1  https://ch.schurter.com/pdf/english/typ_SP_5x2...   
2  https://ch.schurter.com/pdf/english/typ_SP_5x2...   
3  https://ch.schurter.com/pdf/english/typ_SP_5x2...   
4  https://ch.schurter.com/pdf/english/typ_SP_5x2...   

                                       TAXONOMY_PATH ESTIMATED_END_OF_LIFE  \
0    Passives :: Circuit Protection Devices :: Fuses                   9.2   
1  Circuit Protection > Overcurrent Protection > ...                   4.3   
2  Circuit Protection > Overcurrent Protection > ...                   4.3   
3  Circuit Protection > Overcurrent Protection > ...                   4.3   
4  Circuit Protection > Overcurrent Protection > ...                   4.3   

  LIFE_CYCLE_STAGE REACH_DATA_URL DATA_SOURCE_ID  
0           Mature           None            IHS  
1           Mature           None             SE  
2           Mature           None             SE  
3           Mature           None             SE  
4           Mature           None             SE

## Store results to S3

In [21]:
df_att.to_csv('/S3/FILES_MT_S3/df_att.csv', sep=';', index=False)

In [22]:
df_master.to_csv('/S3/FILES_MT_S3/df_master.csv', sep=';', index=False)

## Build ABT 
Here every attribute is going to be on columns for every part ID, that means partnumber is the unique key right now. In addition the part description is concatenated since part description is the best filled "attribute" in SE and IHS

### Creating SE attribute tables

In [28]:
df_att = pd.read_csv('/S3/FILES_MT_S3/df_att.csv', sep=';') 
df_att.head(20)

PART_ID             ATTRIBUTE_NAME  \
0      00012520                     Acting   
1      00012520                Application   
2      00012520             Current Rating   
3      00012520              Fuse Material   
4      00012520  Maximum AC Voltage Rating   
5      00012520        Package - Packaging   
6      00012520                       Type   
7      00012520             Voltage Rating   
8   00225001MXP                     Acting   
9   00225001MXP                Application   
10  00225001MXP             Current Rating   
11  00225001MXP              Fuse Material   
12  00225001MXP  Maximum AC Voltage Rating   
13  00225001MXP        Package - Packaging   
14  00225001MXP           Package - Suffix   
15  00225001MXP       Packaging - Quantity   
16  00225001MXP                       Type   
17  00225001MXP        Typical Melting I2t   
18  00225001MXP             Voltage Rating   
19  00225002MXP                     Acting   

                                      ATTRIBUTE_VALUE ATTRIBUTE_UOM  
0                                           Slow Blow           NaN  
1   Power Supply Adapter|Primary Protection on PCB...           NaN  
2                                                 2.5             A  
3                                             Ceramic           NaN  
4                                                 250             V  
5                                                 Box           NaN  
6                                           Miniature           NaN  
7                                                 250             V  
8                                                Fast           NaN  
9                                       Motor Circuit           NaN  
10                                                  1             A  
11                                              Glass           NaN  
12                                                250             V  
13                                               Bulk           NaN  
14                                                 MX           NaN  
15                                               1000           NaN  
16                                       Subminiature           NaN  
17                                               2.22           A²s  
18                                                250             V  
19                                               Fast           NaN

In [33]:
df_att['ATTRIBUTE_UOM'].fillna(value= '', inplace = True)

In [36]:
#df_att["VALUE"] = df_att['ATTRIBUTE_VALUE'] + "" + df_att['ATTRIBUTE_UOM']
df_att.drop(columns = ['ATTRIBUTE_UOM', 'ATTRIBUTE_VALUE'], inplace = True)

In [37]:
df_att.head()

PART_ID             ATTRIBUTE_NAME  \
0  00012520                     Acting   
1  00012520                Application   
2  00012520             Current Rating   
3  00012520              Fuse Material   
4  00012520  Maximum AC Voltage Rating   

                                               VALUE  
0                                          Slow Blow  
1  Power Supply Adapter|Primary Protection on PCB...  
2                                               2.5A  
3                                            Ceramic  
4                                               250V

In [38]:
df_master_att = create_att_table(df_att,["PART_ID","ATTRIBUTE_NAME"])

In [39]:
df_master_att.shape

(297768, 2617)

In [13]:
df_master = pd.read_csv('/S3/FILES_MT_S3/df_master.csv', sep=';', usecols = ['PART_DESCRIPTION', 'PART_ID', "DATA_SOURCE_ID"] )

In [40]:
df_tmp_SE = df_master.loc[df_master.DATA_SOURCE_ID == 'SE']
df_tmp_SE = df_tmp_SE.fillna(value='').set_index("PART_ID").drop(columns = ["DATA_SOURCE_ID"])
print(df_tmp_SE.shape)
#df_tmp_SE.drop_duplicates(inplace = True)
#print(df_tmp_SE.shape)

(397985, 10)


In [41]:
df_tmp_IHS = df_master[['PART_DESCRIPTION', 'PART_ID', "DATA_SOURCE_ID"]].loc[df_master.DATA_SOURCE_ID == 'IHS']
df_tmp_IHS = df_tmp_IHS.fillna(value='').set_index("PART_ID").drop(columns = ["DATA_SOURCE_ID"])

In [42]:
df_description = (
    df_tmp_SE['PART_DESCRIPTION']
    .to_frame()
    .join(df_tmp_IHS['PART_DESCRIPTION'].to_frame(), how = 'outer', lsuffix='_SE', rsuffix='_IHS')
)

In [47]:
df_description['PART_DESCRIPTION'] = df_description["PART_DESCRIPTION_SE"] + " " + df_description["PART_DESCRIPTION_IHS"]

In [49]:
df_description.drop(["PART_DESCRIPTION_SE","PART_DESCRIPTION_IHS"], inplace=True, axis = 1)
df_description.shape

(1201901, 1)

In [50]:
#df_description.drop_duplicates(inplace = True)
df_description_unique = df_description["PART_DESCRIPTION"].groupby(df_description.index).agg(lambda char_col: Counter(char_col).most_common(1)[0][0])
#df_description = df_description[~df_description.index.duplicated(keep='last')]
df_description_unique.shape

(305171,)

In [51]:
df_description_unique.head()

PART_ID
00011006    Fuse Miniature Fast Acting 1.6A 250V Holder Ca...
00011012    Non Resettable Fuses Electric Fuse, Very Fast ...
00011013    Fuse Miniature Fast Acting 8A 250V Holder Cart...
00011014    Non Resettable Fuses Electric Fuse, Very Fast ...
00011015    Fuse Miniature Fast Acting 12.5A 250V Holder C...
Name: PART_DESCRIPTION, dtype: object

In [55]:
df_ABT = df_description_unique.to_frame().join(df_master_att , how = 'left')

In [56]:
df_ABT.dropna(axis = 1, how = 'all', inplace = True)
df_ABT.shape

(305171, 2618)

## Cleaning column names and Harmonize Categories

In [57]:
cols_cleaned = [col.strip() for col in df_ABT.columns]
df_ABT.columns = cols_cleaned

In [58]:
df_ABT.shape

(305171, 2618)

Combine all the Categories which have equal or less than 10 occurences into a group called "Others"

In [59]:
categories = [
'Relay',
'Resistor',
'Switch',
'Capacitor',
#'Zener',
'Diodes',
#'Suppression', # zu Diodes
'Thyristor',
'Transistor',
#'Transistor FET',
'Oscillator',
#    'Quartz', # mit Oscillators zusammenführen 
'Varistor',
'Fuse',
'LED',
'OptoCoupler',
#'Analog', # Analog zu circuits zusammenführen
'Circuit',
'Controller', 
#'Amplifier', # zur gruppe Analog
'Reactor',
'Filter',
'Transformer',
'Connector', # evtl nur Connector verwenden
#'Micro Controller',
#'Logic Circuit',
#'Switch',
"Cable",
'Memory',
"Regulator", # zu Analog
'Screw']

In [60]:
lst = [re.sub("\W+", " ", col.lower()) for col in categories if len(col) > 2]
#lst = [re.sub("el ", "", col.lower()) for col in lst]
#lst = [re.sub("ic ", "", col.lower()) for col in lst]
#lst = [re.sub("on ", "", col.lower()) for col in lst]
#lst = [re.sub(Electromechanical )]
lst = [col.strip() for col in lst]
category = sorted(lst, key=str.lower)
#category.append('other')
category

['cable',
 'capacitor',
 'circuit',
 'connector',
 'controller',
 'diodes',
 'filter',
 'fuse',
 'led',
 'memory',
 'optocoupler',
 'oscillator',
 'reactor',
 'regulator',
 'relay',
 'resistor',
 'screw',
 'switch',
 'thyristor',
 'transformer',
 'transistor',
 'varistor']

In [61]:
df_master["Cat"] = "999999"
df_tmp = (
    df_master
    .CATEGORY_NAME
    .apply(lambda x: x if type(x)!=str else re.sub("\W+", " " ,x))
    .apply(lambda x: x if type(x)!=str else re.sub("\s+"," " ,x))
    .apply(lambda x: x if type(x)!=str else x.lower())
    .str
    .strip()
)

In [62]:
base = r'^{}'
expr = '(?=.*{})'
for cat in category:
    #print(cat)
    
    words = cat.split()
    # If any 
    tmp = np.where(df_tmp.str.contains(base.format(''.join(expr.format(w) for w in words))), cat,"999999")
    #tmp = np.where(df_tmp.str.contains(cat[:7]), cat,"nan")
    df_master["Cat"] = np.where(df_master["Cat"] == "999999", tmp, df_master["Cat"] )

In [63]:
df_cat = df_master.groupby("PART_ID").agg({'Cat': 'max'})

In [64]:
df_CATEGORY_NAME = df_master.groupby("PART_ID").agg({'CATEGORY_NAME': 'max'})

In [65]:
df_CATEGORY_NAME_ABT =  df_CATEGORY_NAME[df_CATEGORY_NAME.index.isin(df_ABT.index)]
df_cat_ABT = df_cat[df_cat.index.isin(df_ABT.index)]
#df_cat_ABT.Cat.replace('999999', np.nan, inplace = True)

In [66]:
df_cat_ABT.Cat = df_cat_ABT.Cat.apply(lambda x: x.title() if str(x) != '999999' else None)

In [67]:
df_ABT["Category"] = df_cat_ABT.Cat.combine_first(df_CATEGORY_NAME_ABT.CATEGORY_NAME)

In [68]:
df_ABT["Category"].loc[df_ABT.Category.str.lower().str.contains("pin")].value_counts()

Pins               8
Mechanical Pins    3
PIN                1
Name: Category, dtype: int64

## Manual mapping of categories 
FET, Varistor, etc. are combined together. The following mapping are aligned with Stefan (4.9.2019):
1. Suppresion -> Diodes
2. Amplifier -> Analog
3. Regulator -> Analog

In [69]:
def word_replacer(df, col, to_replace, value):
    """
        Fucntion for replacing predefined to_replace list with corresponding values
    """
    
    
    
    n = len(value)
    if n > 1 and len(to_replace) != n:
        print("Error: multiple values for to_replace and values with different length. Either value is one value or list with equal length ")
        return(df)
    else: 
        
        zip_list = zip(to_replace, cycle(value)) if len(to_replace) > len(value) else zip(cycle(to_replace), value)
        for pattern in zip_list:
            df[col].replace(pattern[0] ,pattern[1], inplace = True) 
            #df[column].replace(pattern[0] ,pattern[1]) 
        return df[col]

In [70]:
to_replace = ['Modul' , 'Adapter','Sensor', 'Fan','Microwave', 'Amp', 'Analog', 'Logic', 'FET', 'IC',"LCD", "Clock", "Interface", "Conn", "Bus", "Transistor", "Regulator", "Suppres", "Quartz", "RAM", "PIN", 'Pin' ]

In [71]:
value_to_replace = [ 'Modul', 'Adapter' , 'Sensor' , 'Fan',"Microwave", "Analog", 'Analog', 'Logic Devices', "Transistor", 'IC',"LCD", "Clock", "Interface", "Connector", "Bus", "Transistor", "Analog", "Diodes", "Quartz", "RAM", 'Pin', 'Pin']

In [72]:
replacer_dict = dict(zip(to_replace, value_to_replace))
replacer_dict

{'Modul': 'Modul',
 'Adapter': 'Adapter',
 'Sensor': 'Sensor',
 'Fan': 'Fan',
 'Microwave': 'Microwave',
 'Amp': 'Analog',
 'Analog': 'Analog',
 'Logic': 'Logic Devices',
 'FET': 'Transistor',
 'IC': 'IC',
 'LCD': 'LCD',
 'Clock': 'Clock',
 'Interface': 'Interface',
 'Conn': 'Connector',
 'Bus': 'Bus',
 'Transistor': 'Transistor',
 'Regulator': 'Analog',
 'Suppres': 'Diodes',
 'Quartz': 'Quartz',
 'RAM': 'RAM',
 'PIN': 'Pin',
 'Pin': 'Pin'}

In [75]:
for cat, value_to_replace in replacer_dict.items():
 #   k = 0
    df_ABT["Category"] = word_replacer(df_ABT, "Category", df_ABT["Category"].loc[df_ABT["Category"].str.contains(cat) == True].unique(),[value_to_replace] )

In [76]:
df_ABT["Category"].loc[df_ABT["Category"].str.contains('Pin')].value_counts()

Pin    12
Name: Category, dtype: int64

In [77]:
df_ABT.shape

(305171, 2618)

## Cleaning and saving final ABT
we drop every col that has only 1 observation and delete all rows with only 1 observations besides objectId and partNumber.
THis reduces the dimesnionality of the final dataset

In [ ]:
df_ABT.to_csv('/S3/FILES_MT_S3/df_ABT.csv', sep=';', index = True)